In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict
import random

class OPEXSimulation:
    def __init__(self, 
                 base_year_data: Dict,
                 target_headcounts: Dict,
                 vacancy_rate: float = 0.05,
                 simulation_runs: int = 10000):
        """
        Initialize OPEX Monte Carlo Simulation
        
        Args:
            base_year_data (dict): Historical data for base year
                {
                    'total_permanent_headcount': int,
                    'total_temporary_headcount': int,
                    'avg_permanent_cost_per_head': float,
                    'avg_temporary_cost_per_head': float,
                    'total_opex': float
                }
            target_headcounts (dict): Planned headcounts for future years
                {
                    2024: {
                        'permanent_target': int,
                        'temporary_target': int
                    },
                    2025: {
                        'permanent_target': int,
                        'temporary_target': int
                    }
                }
            vacancy_rate (float): Expected vacancy rate (default 5%)
            simulation_runs (int): Number of Monte Carlo simulation runs
        """
        self.base_year_data = base_year_data
        self.target_headcounts = target_headcounts
        self.vacancy_rate = vacancy_rate
        self.simulation_runs = simulation_runs
        
        # Cost variation parameters (to be estimated)
        self.permanent_cost_var = 0.1  # 10% variation
        self.temporary_cost_var = 0.15  # 15% variation
        self.hiring_month_var = 0.25  # Variance in hiring timing
    
    def generate_hiring_months(self, target_headcount: int) -> List[int]:
        """
        Simulate random months for new hires
        
        Args:
            target_headcount (int): Number of new hires
        
        Returns:
            List of month indices (0-11) for new hires
        """
        # Use a random distribution with higher probability in middle months
        months = np.random.beta(5, 5, target_headcount) * 12
        return [int(month) for month in months]
    
    def calculate_bow_wave_impact(self, new_hires: int, avg_cost_per_head: float, hiring_months: List[int]) -> float:
        """
        Calculate bow wave impact based on hiring timing
        
        Args:
            new_hires (int): Number of new hires
            avg_cost_per_head (float): Average cost per head
            hiring_months (List[int]): Months when hires occur
        
        Returns:
            Additional annual cost due to bow wave effect
        """
        # Weigh additional costs based on when hires occur
        months_worked_multiplier = [(12 - month)/12 for month in hiring_months]
        return new_hires * avg_cost_per_head * sum(months_worked_multiplier) / len(months_worked_multiplier)
    
    def run_simulation(self) -> Dict:
        """
        Run Monte Carlo simulation for OPEX projection
        
        Returns:
            Dictionary with simulation results
        """
        simulation_results = {
            'permanent_opex': [],
            'temporary_opex': [],
            'total_opex': []
        }
        
        for _ in range(self.simulation_runs):
            # Simulate permanent headcount costs
            perm_hired = max(0, self.target_headcounts[2024]['permanent_target'] - 
                             int(self.base_year_data['total_permanent_headcount'] * (1 - self.vacancy_rate)))
            perm_hiring_months = self.generate_hiring_months(perm_hired)
            
            permanent_base_cost = (
                self.base_year_data['total_permanent_headcount'] * 
                self.base_year_data['avg_permanent_cost_per_head']
            )
            
            permanent_var_cost = permanent_base_cost * np.random.normal(1, self.permanent_cost_var)
            permanent_bow_wave = self.calculate_bow_wave_impact(
                perm_hired, 
                self.base_year_data['avg_permanent_cost_per_head'],
                perm_hiring_months
            )
            permanent_total = permanent_var_cost + permanent_bow_wave
            
            # Simulate temporary headcount costs
            temp_hired = max(0, self.target_headcounts[2024]['temporary_target'] - 
                             int(self.base_year_data['total_temporary_headcount'] * (1 - self.vacancy_rate)))
            temp_hiring_months = self.generate_hiring_months(temp_hired)
            
            temporary_base_cost = (
                self.base_year_data['total_temporary_headcount'] * 
                self.base_year_data['avg_temporary_cost_per_head']
            )
            
            temporary_var_cost = temporary_base_cost * np.random.normal(1, self.temporary_cost_var)
            temporary_bow_wave = self.calculate_bow_wave_impact(
                temp_hired, 
                self.base_year_data['avg_temporary_cost_per_head'],
                temp_hiring_months
            )
            temporary_total = temporary_var_cost + temporary_bow_wave
            
            # Aggregate results
            simulation_results['permanent_opex'].append(permanent_total)
            simulation_results['temporary_opex'].append(temporary_total)
            simulation_results['total_opex'].append(permanent_total + temporary_total)
        
        return {
            'permanent_opex_stats': {
                'mean': np.mean(simulation_results['permanent_opex']),
                'median': np.median(simulation_results['permanent_opex']),
                '5th_percentile': np.percentile(simulation_results['permanent_opex'], 5),
                '95th_percentile': np.percentile(simulation_results['permanent_opex'], 95)
            },
            'temporary_opex_stats': {
                'mean': np.mean(simulation_results['temporary_opex']),
                'median': np.median(simulation_results['temporary_opex']),
                '5th_percentile': np.percentile(simulation_results['temporary_opex'], 5),
                '95th_percentile': np.percentile(simulation_results['temporary_opex'], 95)
            },
            'total_opex_stats': {
                'mean': np.mean(simulation_results['total_opex']),
                'median': np.median(simulation_results['total_opex']),
                '5th_percentile': np.percentile(simulation_results['total_opex'], 5),
                '95th_percentile': np.percentile(simulation_results['total_opex'], 95)
            },
            'raw_simulation_data': simulation_results
        }
    
    def plot_simulation_results(self, results: Dict):
        """
        Visualize Monte Carlo simulation results
        
        Args:
            results (Dict): Simulation results from run_simulation()
        """
        plt.figure(figsize=(15, 5))
        
        # Permanent OPEX Distribution
        plt.subplot(1, 3, 1)
        plt.hist(results['raw_simulation_data']['permanent_opex'], bins=50, alpha=0.7)
        plt.title('Permanent OPEX Distribution')
        plt.xlabel('Cost')
        plt.ylabel('Frequency')
        
        # Temporary OPEX Distribution
        plt.subplot(1, 3, 2)
        plt.hist(results['raw_simulation_data']['temporary_opex'], bins=50, alpha=0.7)
        plt.title('Temporary OPEX Distribution')
        plt.xlabel('Cost')
        plt.ylabel('Frequency')
        
        # Total OPEX Distribution
        plt.subplot(1, 3, 3)
        plt.hist(results['raw_simulation_data']['total_opex'], bins=50, alpha=0.7)
        plt.title('Total OPEX Distribution')
        plt.xlabel('Cost')
        plt.ylabel('Frequency')
        
        plt.tight_layout()
        plt.show()

def main():
    # Example usage
    base_year_data = {
        'total_permanent_headcount': 500,
        'total_temporary_headcount': 50,
        'avg_permanent_cost_per_head': 100000,
        'avg_temporary_cost_per_head': 80000,
        'total_opex': 55000000
    }
    
    target_headcounts = {
        2024: {
            'permanent_target': 600,
            'temporary_target': 75
        },
        2025: {
            'permanent_target': 750,
            'temporary_target': 100
        }
    }
    
    sim = OPEXSimulation(base_year_data, target_headcounts)
    results = sim.run_simulation()
    
    # Print detailed statistics
    print("Permanent OPEX Statistics:")
    for key, value in results['permanent_opex_stats'].items():
        print(f"{key}: {value:,.2f}")
    
    print("\nTemporary OPEX Statistics:")
    for key, value in results['temporary_opex_stats'].items():
        print(f"{key}: {value:,.2f}")
    
    print("\nTotal OPEX Statistics:")
    for key, value in results['total_opex_stats'].items():
        print(f"{key}: {value:,.2f}")
    
    # Optional: Visualize results
    sim.plot_simulation_results(results)

if __name__ == "__main__":
    main()